<a href="https://colab.research.google.com/github/fouad89/arabicNLP/blob/main/fine_tuning_Arabert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.0 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
     |████████████████████████████████| 596 kB 69.3 MB/s 
     |████████████████████████████████| 6.6 MB 37.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install farasapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!git clone https://github.com/aub-mind/arabert.git

Cloning into 'arabert'...
remote: Enumerating objects: 569, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 569 (delta 26), reused 24 (delta 23), pack-reused 535
Receiving objects: 100% (569/569), 9.12 MiB | 31.04 MiB/s, done.
Resolving deltas: 100% (327/327), done.


In [4]:
!pip install pyarabic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 126 kB 4.5 MB/s 


In [24]:
!pip install fast_ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 285 kB/s 


## sentiment using pipeline

In [5]:
# using pipeline
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

In [6]:
# arabert model from hugging face https://huggingface.co/aubmindlab/bert-large-arabertv2
model_name = "aubmindlab/bert-large-arabertv2"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38G [00:00<?, ?B/s]

Some weights of the model checkpoint at aubmindlab/bert-large-arabertv2 were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

Downloading:   0%|          | 0.00/611 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

## Using Arabert Preprocessor

In [7]:
from arabert.preprocess import ArabertPreprocessor
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [8]:
arabert_prep = ArabertPreprocessor(model_name=model_name)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


100%|██████████| 241M/241M [00:20<00:00, 12.0MiB/s]


[2022-06-02 14:29:29,883 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [9]:
!git clone https://github.com/fouad89/arabicNLP

Cloning into 'arabicNLP'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 28 (delta 10), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (28/28), done.


In [10]:
os.listdir("arabicNLP/data")

['Tweets.txt']

In [11]:
data_dir = "arabicNLP/data/Tweets.txt"
data = pd.read_csv(data_dir, sep='\t', header=None)
data.columns = ['text', 'label']

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9694 entries, 0 to 9693
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    9694 non-null   object
 1   label   9694 non-null   object
dtypes: object(2)
memory usage: 151.6+ KB


In [13]:
data['label'].value_counts()

OBJ        6470
NEG        1642
NEUTRAL     805
POS         777
Name: label, dtype: int64

## Fine-tuning Process
<ol>
<li>Prepare Dataset</li>
<li>Load pretrained tokenizer</li>
<li>Build Pytorch dataset with encodings</li>
<li>load trainer and train</li>

</ol>

In [25]:
import torch
from torch.utils.data import Dataset
from transformers import Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
from fast_ml.model_development import train_valid_test_split


In [19]:
# prepare dataset
texts = data['text'].values
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(data['label'])
# labels = data['label'].values

In [20]:
encoded_labels

array([2, 3, 0, ..., 2, 3, 0])

In [21]:
label_encoder.classes_

array(['NEG', 'NEUTRAL', 'OBJ', 'POS'], dtype=object)

In [22]:
label_encoder.inverse_transform([1])

array(['NEUTRAL'], dtype=object)